In [ ]:
import pandas as pd
import networkx as nx
import numpy as np

## Pre-processing

In [1]:
df = pd.read_csv('usa_airport/usa_airport.csv')
df.head(5)

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.254101,-121.150002
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.254101,-121.150002


In [2]:
# 3606803 records
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3606803 entries, 0 to 3606802
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Origin_airport          object 
 1   Destination_airport     object 
 2   Origin_city             object 
 3   Destination_city        object 
 4   Passengers              int64  
 5   Seats                   int64  
 6   Flights                 int64  
 7   Distance                int64  
 8   Fly_date                object 
 9   Origin_population       int64  
 10  Destination_population  int64  
 11  Org_airport_lat         float64
 12  Org_airport_long        float64
 13  Dest_airport_lat        float64
 14  Dest_airport_long       float64
dtypes: float64(4), int64(6), object(5)
memory usage: 412.8+ MB


In [3]:
# Change object type to string
df['Origin_airport'] = df['Origin_airport'].astype('str')
df['Destination_airport'] = df['Destination_airport'].astype('str')
df['Origin_city'] = df['Origin_city'].astype('str')
df['Destination_city'] = df['Destination_city'].astype('str')
df['Fly_date'] = df['Fly_date'].astype('str')

In [4]:
df.describe()

,Passengers,Seats,Flights,Distance,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
count,3.606803e+06,3.606803e+06,3.606803e+06,3.606803e+06,3.606803e+06,3.606803e+06,3.599849e+06,3.599849e+06,3.599996e+06,3.599996e+06
mean,2.688910e+03,4.048297e+03,3.722890e+01,6.973190e+02,5.871502e+06,5.897982e+06,3.775029e+01,-9.186178e+01,3.774091e+01,-9.183433e+01
std,4.347617e+03,6.200871e+03,4.961970e+01,6.044165e+02,7.858062e+06,7.906127e+06,5.765453e+00,1.653773e+01,5.736556e+00,1.647228e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.300500e+04,1.288700e+04,1.972140e+01,-1.579220e+02,1.972140e+01,-1.579220e+02
25%,1.090000e+02,1.560000e+02,2.000000e+00,2.730000e+02,1.030597e+06,1.025470e+06,3.363670e+01,-9.703800e+01,3.363670e+01,-9.703800e+01
50%,1.118000e+03,1.998000e+03,2.500000e+01,5.190000e+02,2.400193e+06,2.400193e+06,3.885210e+01,-8.775240e+01,3.885210e+01,-8.775240e+01
75%,3.503000e+03,5.370000e+03,5.500000e+01,9.270000e+02,8.613622e+06,8.635706e+06,4.173260e+01,-8.029060e+01,4.173260e+01,-8.029060e+01
max,8.959700e+04,1.470620e+05,1.128000e+03,5.095000e+03,3.813959e+07,3.813959e+07,6.483750e+01,-6.882810e+01,6.483750e+01,-6.882810e+01


In [5]:
# long and lat is missing
df.isnull().sum()

Origin_airport               0
Destination_airport          0
Origin_city                  0
Destination_city             0
Passengers                   0
Seats                        0
Flights                      0
Distance                     0
Fly_date                     0
Origin_population            0
Destination_population       0
Org_airport_lat           6954
Org_airport_long          6954
Dest_airport_lat          6807
Dest_airport_long         6807
dtype: int64

In [6]:
# Original: Origin_airport + Origin_city + Org_airport_lat + Org_airport_long
# Dest: Destination_airport + Destination_city + Dest_airport_lat + Dest_airport_long
# Info (in a date): Passengers + Seats + Flights + Distance + Fly_date
# Fluctuate: Origin_population + Destination_population (by date) + Fly_date

## Airport Processing

In [7]:
# Make sure it mapping:
df['Origin_airport'].nunique() #683

683

In [8]:
df['Destination_airport'].nunique() #708

708

In [9]:
is_origin_mapping = df[['Origin_airport','Origin_city','Org_airport_lat','Org_airport_long']].drop_duplicates().shape[0] == df['Origin_airport'].nunique()
if is_origin_mapping:
    print("There original is mapping") #683

There original is mapping


In [10]:
is_dest_mapping = df[['Destination_airport','Destination_city','Dest_airport_lat','Dest_airport_long']].drop_duplicates().shape[0] == df['Destination_airport'].nunique()
if is_dest_mapping:
    print("There destination is mapping") #708

There destination is mapping


In [11]:
df_origin = df[['Origin_airport','Origin_city','Org_airport_lat','Org_airport_long']].drop_duplicates() #683
df_dest = df[['Destination_airport','Destination_city','Dest_airport_lat','Dest_airport_long']].drop_duplicates() #708


In [12]:
rename_origin_dict = {
    'Origin_airport': 'airport',
    'Origin_city': 'city',
    'Org_airport_lat': 'latitude',
    'Org_airport_long': 'longitude'
}
rename_dest_dict = {
    'Destination_airport': 'airport',
    'Destination_city': 'city',
    'Dest_airport_lat': 'latitude',
    'Dest_airport_long': 'longitude'
}

df_origin_rename = df_origin.rename(columns= rename_origin_dict)
df_dest_rename = df_dest.rename(columns= rename_dest_dict)

In [13]:
df_origin_rename.isnull().sum()

airport        0
city           0
latitude     206
longitude    206
dtype: int64

In [14]:
df_dest_rename.isnull().sum()

airport        0
city           0
latitude     224
longitude    224
dtype: int64

In [15]:
list_column_airport = ['airport','city','latitude','longitude']

# Left: 683 - Right: 708
# Outer : 728
# Inner : 664
print(f"Left DF: {df_origin_rename.shape[0]} records")
print(f"Right DF: {df_dest_rename.shape[0]} records")

df_join_outer = pd.merge(df_origin_rename, df_dest_rename, on=list_column_airport, how='outer')
print(f"Outer join: {df_join_outer.shape[0]} records")

df_join_inner = pd.merge(df_origin_rename, df_dest_rename, on=list_column_airport, how='inner')
print(f"Inner join: {df_join_inner.shape[0]} records")

print(f">> Anti left: {df_origin_rename.shape[0]-df_join_inner.shape[0]}")
print(f">> Anti right: {df_dest_rename.shape[0]-df_join_inner.shape[0]}")

Left DF: 683 records
Right DF: 708 records
Outer join: 727 records
Inner join: 664 records
>> Anti left: 19
>> Anti right: 44


In [16]:
df_airports = df_join_outer
df_airports

,airport,city,latitude,longitude
0,MHK,"Manhattan, KS",39.140999,-96.670799
1,EUG,"Eugene, OR",44.124599,-123.211998
2,MFR,"Medford, OR",42.374199,-122.873001
3,SEA,"Seattle, WA",47.449001,-122.308998
4,PDX,"Portland, OR",45.588699,-122.598000
...,...,...,...,...
722,MIW,"Marshalltown, IA",NaN,NaN
723,JCC,"San Francisco, CA",NaN,NaN
724,NGP,"Corpus Christi, TX",NaN,NaN
725,MPS,"Mount Pleasant, TX",NaN,NaN


In [17]:
# 206/683 + 224/708
df_airports.isnull().sum() #243/728

airport        0
city           0
latitude     243
longitude    243
dtype: int64

In [18]:
df_airports.to_csv('usa_airport/airports.csv', index=False, header=True)

## Distances processing

In [19]:
# Get columns Org, Dest airport id and their distances
df_distances = df[['Origin_airport','Destination_airport','Distance']]
df_distances

,Origin_airport,Destination_airport,Distance
0,MHK,AMW,254
1,EUG,RDM,103
2,EUG,RDM,103
3,EUG,RDM,103
4,MFR,RDM,156
...,...,...,...
3606798,STL,TBN,119
3606799,STL,TBN,119
3606800,STL,TBN,119
3606801,CGI,TBN,146


In [20]:
# See the df when source and destination are the same
df_same = df_distances.loc[df_distances['Origin_airport'] == df_distances['Destination_airport']]
df_same

,Origin_airport,Destination_airport,Distance
1025,RDM,RDM,0
4217,EKO,EKO,0
4239,EKO,EKO,0
5052,EKO,EKO,0
5548,EKO,EKO,0
...,...,...,...
3602337,COS,COS,0
3604074,COS,COS,0
3604812,COS,COS,0
3606397,HII,HII,0


In [21]:
# There is no special or wrong distance value when source and destination are the same
df_same_anomaly = df_same.loc[df_same['Distance'] > 0]
len(df_same_anomaly)

0

In [22]:
df_distances_diff = df_distances.loc[df_distances['Origin_airport'] != df_distances['Destination_airport']]
df_distances_diff = df_distances_diff.drop_duplicates() # Remove the duplicates
df_distances_diff

,Origin_airport,Destination_airport,Distance
0,MHK,AMW,254
1,EUG,RDM,103
4,MFR,RDM,156
9,SEA,RDM,228
14,PDX,RDM,116
...,...,...,...
3606757,MWA,TBN,171
3606770,MCI,TBN,176
3606792,BRL,TBN,217
3606801,CGI,TBN,146


In [23]:
# # Due to Org-Dest and Dest-Org is the same distance journey. We try to swap it
# # Swap values of A and B if A > B
# mask = df_flights['Origin_airport'] > df_flights['Destination_airport']
# df_flights.loc[mask, ['Origin_airport', 'Destination_airport']] = df_flights.loc[mask, ['Destination_airport', 'Origin_airport']].values
# df_flights

In [24]:
# Check the length of dataframe with and without distance
print("Number of distinct records in distances:", len(df_distances_diff))
df_org_dest = df_distances_diff[['Origin_airport','Destination_airport']]
print("Number of distinct records in org-dest pair:", len(df_org_dest.drop_duplicates()))
# So there are existing records with the same org-dest pair but different distances

Number of distinct records in distances: 38158
Number of distinct records in org-dest pair: 36449


In [25]:
# Count the frequency of org-dest with distance value
df_distances_count = df_distances_diff.groupby(['Origin_airport','Destination_airport']).size().reset_index(name="count_dist").sort_values("count_dist",ascending=False)
df_distances_count

,Origin_airport,Destination_airport,count_dist
2312,AUS,LRD,4
32271,SJC,AUS,4
26758,PDX,AUS,4
20430,LRD,AUS,4
32859,SMF,AUS,4
...,...,...,...
12483,FLL,RDU,1
12482,FLL,RDM,1
12481,FLL,RAP,1
12480,FLL,PWM,1


In [26]:
df_distances_count_count = df_distances_count.groupby('count_dist').size().reset_index(name='count')
df_distances_count_count

,count_dist,count
0,1,34836
1,2,1524
2,3,82
3,4,7


In [27]:
# Let check when org-dest pair when it have 4 different distances
df_distances_tmp = df_distances_count[df_distances_count['count_dist']==4].head(1)
df_distances_tmp

,Origin_airport,Destination_airport,count_dist
2312,AUS,LRD,4


In [28]:
# Check when Origin_airport = AUS and Destination_airport = LRD and otherwise
df_distances_tmp = df_distances_diff.loc[ ( (df_distances_diff['Origin_airport'] == 'AUS') & (df_distances_diff['Destination_airport'] == 'LRD'))
                                         | ((df_distances_diff['Destination_airport'] == 'AUS') & (df_distances_diff['Origin_airport'] == 'LRD')) ]
df_distances_tmp

,Origin_airport,Destination_airport,Distance
193446,LRD,AUS,218
196365,LRD,AUS,212
198050,LRD,AUS,216
198166,LRD,AUS,211
436501,AUS,LRD,218
437238,AUS,LRD,216
437244,AUS,LRD,212
437566,AUS,LRD,211


In [29]:
df_distances_minmax = df_distances_diff.groupby(['Origin_airport','Destination_airport']).agg(
    Distance_min=pd.NamedAgg(column="Distance", aggfunc="min"),
    Distance_max=pd.NamedAgg(column="Distance", aggfunc="max"),
    Distance_mean=pd.NamedAgg(column="Distance", aggfunc="mean"),
    Distance_count=pd.NamedAgg(column="Distance", aggfunc="count")).reset_index()
df_distances_minmax['Distance_diff'] = df_distances_minmax['Distance_max'] - df_distances_minmax['Distance_min']
df_distances_minmax = df_distances_minmax.sort_values('Distance_diff')
df_distances_minmax

,Origin_airport,Destination_airport,Distance_min,Distance_max,Distance_mean,Distance_count,Distance_diff
0,1B1,BDL,58,58,58.0,1,0
24145,MSO,BGR,2158,2158,2158.0,1,0
24144,MSO,BFL,833,833,833.0,1,0
24143,MSO,BFI,389,389,389.0,1,0
24142,MSO,BDL,2053,2053,2053.0,1,0
...,...,...,...,...,...,...,...
23708,MMI,BRO,0,1005,502.5,2,1005
23713,MMI,LRD,0,1031,515.5,2,1031
20552,LRD,MMI,0,1031,515.5,2,1031
18610,LAS,AFW,0,1038,519.0,2,1038


In [30]:
df_distances_minmax_tmp = df_distances_minmax.loc[df_distances_minmax['Distance_diff'] > 40]
df_distances_minmax_tmp

,Origin_airport,Destination_airport,Distance_min,Distance_max,Distance_mean,Distance_count,Distance_diff
1973,ATL,MCN,34,79,56.500000,2,45
21461,MCN,ATL,34,79,56.500000,2,45
12167,FAY,MCN,317,362,339.500000,2,45
25230,OAJ,MCN,377,431,404.000000,2,54
12553,FLO,MCN,249,306,277.500000,2,57
16947,ILM,MCN,349,410,379.500000,2,61
427,ABY,MCN,86,154,120.000000,2,68
24976,MYR,MCN,282,350,316.000000,2,68
33493,STL,OAK,1725,1795,1748.666667,3,70
15065,GTR,MCN,207,291,249.000000,2,84


In [31]:
df_distances_minmax.groupby('Distance_diff').size()

Distance_diff
0       34836
1        1097
2          79
3          70
4          45
5          36
6          49
7          62
8          68
9          15
10          6
11          5
12          8
13          2
14          4
15         15
16          4
17          4
18          3
19         10
20          2
21          3
30          2
31          1
45          3
54          1
57          1
61          1
68          2
70          1
84          1
89          2
95          1
96          1
103         1
105         1
280         1
307         1
1005        1
1031        2
1038        1
1279        1
dtype: int64

In [32]:
# Get the mean if the Distance_diff < 200, Get max if > 200
df_distances_processed = df_distances_minmax
df_distances_processed['Distance'] = np.where(df_distances_processed['Distance_diff'] < 200, df_distances_processed['Distance_mean'], df_distances_processed['Distance_max'])
df_distances_processed = df_distances_processed[['Origin_airport','Destination_airport','Distance']]
df_distances_processed

,Origin_airport,Destination_airport,Distance
0,1B1,BDL,58.0
24145,MSO,BGR,2158.0
24144,MSO,BFL,833.0
24143,MSO,BFI,389.0
24142,MSO,BDL,2053.0
...,...,...,...
23708,MMI,BRO,1005.0
23713,MMI,LRD,1031.0
20552,LRD,MMI,1031.0
18610,LAS,AFW,1038.0


In [33]:
df_distances_processed.to_csv('usa_airport/distances.csv', index=False, header=True)

## Seats & Passengers & Flights processing

In [34]:
df_info = df[['Origin_airport','Destination_airport','Seats','Passengers','Flights']]
df_info

,Origin_airport,Destination_airport,Seats,Passengers,Flights
0,MHK,AMW,30,21,1
1,EUG,RDM,396,41,22
2,EUG,RDM,342,88,19
3,EUG,RDM,72,11,4
4,MFR,RDM,18,0,1
...,...,...,...,...,...
3606798,STL,TBN,969,281,51
3606799,STL,TBN,1026,245,54
3606800,STL,TBN,1273,363,67
3606801,CGI,TBN,19,2,1


In [35]:
# So there are some flights that illegal due to #seats < #passengers
print('Number of records:', len(df_info))
df_info_filter = df_info.loc[df_info['Seats'] < df_info['Passengers']]
print('Number of records when #seats < #passengers:', len(df_info_filter))
df_info_filter

Number of records: 3606803
Number of records when #seats < #passengers: 280


,Origin_airport,Destination_airport,Seats,Passengers,Flights
15362,LAS,RNO,163,179,1
52694,TPA,MIA,179,250,1
56753,JFK,MIA,680,681,4
58768,JFK,MIA,137,167,1
58833,JFK,MIA,137,158,1
...,...,...,...,...,...
3433918,JFK,SFO,197,198,1
3444408,ORD,SFO,163,179,1
3444744,SMF,SFO,163,191,1
3445376,HNL,SFO,376,379,1


In [36]:
df_info.loc[df_info['Flights'] == 0]

,Origin_airport,Destination_airport,Seats,Passengers,Flights
301,SEA,RDM,0,0,0
508,SEA,RDM,0,0,0
9622,TWF,EKO,0,0,0
9777,ITH,ERI,0,0,0
10013,PIT,ERI,0,0,0
...,...,...,...,...,...
3602551,CVG,COS,0,0,0
3602586,ORD,COS,0,0,0
3603050,CVG,COS,0,0,0
3603057,CVG,COS,0,0,0


In [37]:
# At this airport: Mistake or some noise reason
df_info.loc[df_info['Origin_airport'] == df_info['Destination_airport']]

,Origin_airport,Destination_airport,Seats,Passengers,Flights
1025,RDM,RDM,0,0,2
4217,EKO,EKO,124,116,1
4239,EKO,EKO,124,100,1
5052,EKO,EKO,248,238,1
5548,EKO,EKO,124,119,1
...,...,...,...,...,...
3602337,COS,COS,0,0,0
3604074,COS,COS,0,0,1
3604812,COS,COS,0,0,1
3606397,HII,HII,0,0,1


In [38]:
# Processing: Just filter different and get sum all passengers and flights for routes between 1990 and 2009
df_info_processed = df_info.loc[df_info['Origin_airport'] != df_info['Destination_airport']]
df_info_processed = df_info_processed.groupby(['Origin_airport','Destination_airport']).agg(
    Passengers_sum=pd.NamedAgg(column="Seats", aggfunc="sum"),
    Flights_sum=pd.NamedAgg(column="Flights", aggfunc="sum")
).reset_index()
df_info_processed

,Origin_airport,Destination_airport,Passengers_sum,Flights_sum
0,1B1,BDL,8,1
1,ABE,ACT,50,1
2,ABE,ACY,103,1
3,ABE,ADS,0,1
4,ABE,AGS,120,2
...,...,...,...,...
36444,YUM,YKM,115,1
36445,ZZV,MCI,0,1
36446,ZZV,PDK,0,5
36447,ZZV,SHV,0,2


In [39]:
df_info_processed.to_csv('usa_airport/info.csv', index=False, header=True)

## Graph analysis with NetworkX

In [103]:
# Create Directed graph
DG = nx.DiGraph()
DG.add_weighted_edges_from(list(df_distances.itertuples(index=False,name=None)), weight='distance')
DG.add_weighted_edges_from(list(df_info[['Origin_airport','Destination_airport','Passengers']].itertuples(index=False,name=None)), weight='passenger')
DG.add_weighted_edges_from(list(df_info[['Origin_airport','Destination_airport','Flights']].itertuples(index=False,name=None)), weight='flight')

In [105]:
# Query 1: Find the number of airports
DG.number_of_nodes()
# DG.edges
DG.nodes

727

In [106]:
# Query 2: Find the number of routes between airports
DG.number_of_edges()

36719

In [111]:
# Query 3: Which airports have most incoming passengers
sorted(DG.in_degree(weight='passenger'), key=lambda x: x[1], reverse=True)[:5]

[('DFW', 642025),
 ('ATL', 489911),
 ('LAS', 407644),
 ('ORD', 356327),
 ('PHX', 320084)]

In [112]:
# Query 4: Which airports have most incoming flights 
sorted(DG.in_degree(weight='flight'), key=lambda x: x[1], reverse=True)[:5]

[('DFW', 8110), ('ATL', 8062), ('IAH', 5933), ('ORD', 5256), ('DTW', 4868)]

In [113]:
# Query 5: Which airports have most outcoming passengers
sorted(DG.out_degree(weight='passenger'), key=lambda x: x[1], reverse=True)[:5]

[('DFW', 595211),
 ('ATL', 563885),
 ('LAS', 423300),
 ('IAH', 336947),
 ('MDW', 313317)]

In [114]:
# Query 6: Which airports have most outcoming flights 
sorted(DG.out_degree(weight='flight'), key=lambda x: x[1], reverse=True)[:5]

[('ATL', 8072), ('DFW', 8012), ('IAH', 6407), ('DTW', 4947), ('ORD', 4942)]

In [126]:
# Query 7: List 10 airports that airport code 'ALO' can have a flight to
list(DG.neighbors('LBL'))[:10]

['EKO', 'WDG', 'HYS', 'RNO', 'YUM', 'ADS', 'PIA', 'PUB', 'SLN', 'FOE']

In [131]:
# Query 8: List 5 most important airports base on passengers according to PageRank
pr = nx.pagerank(DG, alpha=0.9, weight='passenger')
sorted(pr.items(), key=lambda x: x[1], reverse=True)[:5]

[('DFW', 0.03723508495492591),
 ('ATL', 0.02980045014712284),
 ('LAS', 0.026902272706925644),
 ('ORD', 0.021042451921218147),
 ('PHX', 0.019407068542008486)]

In [132]:
# Query 9: List 5 most important airports base on flights according to PageRank
pr = nx.pagerank(DG, alpha=0.9, weight='flight')
sorted(pr.items(), key=lambda x: x[1], reverse=True)[:5]

[('ATL', 0.02790723383705421),
 ('DFW', 0.02741464996774892),
 ('IAH', 0.020300221804686557),
 ('ORD', 0.018001343607215326),
 ('MEM', 0.016850355854139378)]

In [156]:
# Query 10: Get the lowest cost and highest cost from airport code HYS
lst = sorted(DG.edges('HYS', data=True), key=lambda x: x[2]['distance'], reverse=True)
print(f"""Max distance is from HYS to {lst[0][1]} with {lst[0][2]['distance']} miles""")
print(f"""Min distance is from HYS to {lst[-1][1]} with {lst[-1][2]['distance']} miles""")


Max distance is from HYS to LIT with 483 miles
Min distance is from HYS to GBD with 41 miles


In [158]:
# Query 11: Get the shortest path and its cost from SLB to CRE
path = nx.shortest_path(DG, source='SLB', target='CRE', weight='distance', method='dijkstra')
print('Shortest path from SLB to CRE:',' -> '.join(path))
def get_cost(path: list[str]) -> int:
    res = 0
    for i in range(len(path)-1):
        res += DG[path[i]][path[i+1]]['distance']
    return res
print('Total cost from SLB to CRE:', get_cost(path), 'miles')

Shortest path from SLB to CRE: SLB -> WFB -> MTN -> FAY -> ACY -> CRE
Total cost from SLB to CRE: 2989 miles


In [161]:
# Query 12: Check the connectitivity of Graph (Weakly connected and Strongly connected)
print("Is the graph weakly connected:", nx.is_weakly_connected(DG))
print("The number of weakly connected components is:", nx.number_weakly_connected_components(DG))
print("Is the graph strongly connected:", nx.is_strongly_connected(DG))
print("The number of strongly connected components is:", nx.number_strongly_connected_components(DG))


Is the graph weakly connected: True
The number of weakly connected components is: 1
Is the graph strongly connected: False
The number of strongly connected components is: 65
